# Multiple Channels One-Shot Side Entanglement Optimization

## Imports

In [70]:
import numpy as np
import random
import itertools
import math
import time
import pickle
from typing import Optional, Tuple, cast, List, Dict
from bitarray.util import count_xor, urandom
from bitarray import frozenbitarray
from qiskit import Aer, QuantumCircuit, execute, QuantumRegister, ClassicalRegister
from qiskit.result.result import Result
from qiskit.aqua.components.optimizers import SLSQP, L_BFGS_B, CRS, DIRECT_L, DIRECT_L_RAND, ESCH, ISRES

## General auxiliary functions

In [71]:
""" save and load results to and from a file """
def save_results_to_disk(obj, name ):
    with open('results/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_results_from_file(name ):
    with open('results/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [72]:
def get_combinations_n_etas_without_repeats(number_channels_to_discriminate: int = 2, eta_partitions: int = 20) -> List[List[float]]:
    """ from a given list of attenuations factors create a
        list of all combinatorial groups of n possible etas
        without repeats
        For us it is the same testing first eta 0.1 and second eta 0.2
        than first eta 0.2 and second eta 0.1
        Though, we will always put the greater value as the first group element
    """
    if number_channels_to_discriminate <= 1:
        raise ValueError('number_channels_to_discriminate must be at least 2')
    etas = np.append(np.arange(0, np.pi/2, np.pi/2/eta_partitions), np.pi/2)
    # get combinations of etas without repeats
    eta_groups = list(itertools.combinations(etas, number_channels_to_discriminate))
    return [sorted(eta_group, reverse=True) for eta_group in eta_groups]

In [136]:
GLOBAL_BACKEND=Aer.get_backend('qasm_simulator')

optimization_setup = { 'optimizer_algorithms': ['CRS'],
                        'optimizer_iterations': [1000],
                        'eta_partitions': 20, # number of partitions for the eta ranging from 0 to pi/2
                        'number_channels_to_discriminate': 2 }
GLOBAL_ETA_GROUPS = get_combinations_n_etas_without_repeats(optimization_setup['number_channels_to_discriminate'],
                                                           optimization_setup['eta_partitions'])
GLOBAL_ETA_GROUP = [0]*optimization_setup['number_channels_to_discriminate']
GLOBAL_PLAYS = 10000
filename='20210413a_C2_A1_1000_10000_2'


## Auxiliary functions for preparing the optimization (until the cost function)

In [74]:
    def _select_eta_groups_to_optimize(clone_setup: Optional[dict], number_groups: int = 2) -> Tuple[List[int], dict]:
        """ from the given clone setup, select the eta groups to be optimized,
            and set the non computed groups configuration as default values   """

        eta_group_idx_init, eta_group_idx_end = _set_eta_group_index_bounds(clone_setup)
        index_dict = _build_eta_group_index_lists(eta_group_idx_init, eta_group_idx_end)
        default_optimal_configurations = _set_default_optimal_configurations(index_dict['eta_group_idx_to_skip'], number_groups)

        return (index_dict['eta_group_idx_to_compute'], default_optimal_configurations)

In [75]:
def _set_default_optimal_configurations(eta_group_idx_to_skip: List[int], number_groups: int = 2) -> dict:
    """ Return the optimal configurations set to default values for the indexes to be skipped """
    elements_to_skip = len(eta_group_idx_to_skip)

    configurations: List[dict] = []
    for eta_group_idx in eta_group_idx_to_skip:
        one_configuration = {'state_probability': 0,
                             'angle_rx1': 0,
                             'angle_ry1': 0,
                             'angle_rx0': 0,
                             'angle_ry0': 0,
                             'eta_group': GLOBAL_ETA_GROUPS[eta_group_idx]}
        configurations.append(one_configuration)

    return {'eta_groups': [],
            'best_algorithm': ['NA'] * elements_to_skip,
            'probabilities': [0] * elements_to_skip,
            'configurations': configurations,
            'number_calls_made': [0] * elements_to_skip}

In [76]:
def _set_eta_group_index_bounds(clone_setup: Optional[dict]) -> Tuple[int, int]:
    """ set the first and last eta pair index from which to optimize the configuration """
    total_eta_groups = len(GLOBAL_ETA_GROUPS)

    if clone_setup is None or clone_setup['total_clones'] <= 1:
        return (0, total_eta_groups)

    eta_group_idx_init = int(np.floor(clone_setup['id_clone'] * total_eta_groups / clone_setup['total_clones']))
    eta_group_idx_end = min(int((clone_setup['id_clone'] + 1) *
                               total_eta_groups / clone_setup['total_clones']), total_eta_groups)
    return (eta_group_idx_init, eta_group_idx_end)

def _build_eta_group_index_lists(eta_group_idx_init: int, eta_group_idx_end: int) -> Dict:
    """ create two lists with the the eta pair index to be computed and the index to be skipped """
    first_part_to_skip = list(range(0, eta_group_idx_init))
    last_part_to_skip = list(range(eta_group_idx_end, len(GLOBAL_ETA_GROUPS)))

    return {
        'eta_group_idx_to_compute': list(range(eta_group_idx_init, eta_group_idx_end)),
        'eta_group_idx_to_skip': first_part_to_skip + last_part_to_skip
    }

In [79]:
def _compute_best_configuration(optimization_setup: dict) -> dict:
    """ Find out the best configuration with a global pair of etas (channels) trying out
        a list of specified optimization algorithm """
    optimizer_algorithms = optimization_setup['optimizer_algorithms']
    optimizer_iterations = optimization_setup['optimizer_iterations']
    best_probability = 0
    best_configuration = []
    best_optimizer_algorithm = ""
    number_calls_made = 0

    for optimizer_algorithm, max_evals in zip(optimizer_algorithms, optimizer_iterations):
        print("Analyzing Optimizer Algorithm: ", optimizer_algorithm)
        if optimizer_algorithm == 'SLSQP':
            optimizer = SLSQP(maxiter=max_evals)
        if optimizer_algorithm == 'L_BFGS_B':
            optimizer = L_BFGS_B(maxfun=max_evals, maxiter=max_evals)
        if optimizer_algorithm == 'CRS':
            optimizer = CRS(max_evals=max_evals)
        if optimizer_algorithm == 'DIRECT_L':
            optimizer = DIRECT_L(max_evals=max_evals)
        if optimizer_algorithm == 'DIRECT_L_RAND':
            optimizer = DIRECT_L_RAND(max_evals=max_evals)
        if optimizer_algorithm == 'ESCH':
            optimizer = ESCH(max_evals=max_evals)
        if optimizer_algorithm == 'ISRES':
            optimizer = ISRES(max_evals=max_evals)

        ret = optimizer.optimize(num_vars=len(optimization_setup['initial_parameters']),
                                 objective_function=_cost_function,
                                 variable_bounds=optimization_setup['variable_bounds'],
                                 initial_point=optimization_setup['initial_parameters'])

        print("Best Average Probability:", -ret[1])
        if (-ret[1]) > best_probability:
            best_configuration = ret[0]
            best_probability = -ret[1]
            number_calls_made = ret[2]
            best_optimizer_algorithm = optimizer_algorithm

    # Print results
    print("Final Best Optimizer Algorithm: ", best_optimizer_algorithm)
    print("Final Best Average Probability:", best_probability)
    print("Number of cost function calls made:", number_calls_made)
    print("Parameters Found: state_probability = " + str(best_configuration[0]) +
          ", " + u"\u03D5" + "rx1 = " + str(int(math.degrees(best_configuration[1]))) + u"\u00B0" +
          ", " + u"\u03D5" + "ry1 = " + str(int(math.degrees(best_configuration[2]))) + u"\u00B0" +
          ", " + u"\u03D5" + "rx0 = " + str(int(math.degrees(best_configuration[3]))) + u"\u00B0" +
          ", " + u"\u03D5" + "ry0 = " + str(int(math.degrees(best_configuration[4]))) + u"\u00B0" +
          ''.join([", " + u"\u03B7" + u"\u2080" + " = " + str(int(math.degrees(global_eta))) + u"\u00B0" for global_eta in GLOBAL_ETA_GROUP]))
    
    best_configuration_dict = {'state_probability': best_configuration[0],
                               'angle_rx1': best_configuration[1],
                               'angle_ry1': best_configuration[2],
                               'angle_rx0': best_configuration[3],
                               'angle_ry0': best_configuration[4],
                               'eta_pair': GLOBAL_ETA_GROUP} 
    
    return {'best_algorithm': best_optimizer_algorithm,
            'best_probability': best_probability,
            'best_configuration': best_configuration_dict,
            'number_calls_made': number_calls_made} 

In [80]:
def _cost_function(params: List[float]) -> float:
    """ Computes the cost of running a specific configuration for the number of plays
        defined in the optimization setup.
        Cost is computed as 1 (perfect probability) - average success probability for
        all the plays with the given configuration
        Returns the Cost (error probability).
    """
    configuration = {
        'state_probability': params[0],
        'angle_rx1': params[1],
        'angle_ry1': params[2],
        'angle_rx0': params[3],
        'angle_ry0': params[4],
        'eta_group': GLOBAL_ETA_GROUP}

    return - compute_average_success_probability(configuration=configuration,
                                                       plays=GLOBAL_PLAYS)

## Auxiliary functions for fast compute the circuit and success average probability

In [133]:
def compute_average_success_probability(configuration: dict,
                                            plays: Optional[int] = 1000,) -> float:
    """ Computes the average success probability of running a specific configuration
        for the number of plays defined in the configuration and for each eta from
        the eta group
    """
    eta_counts = [cast(Result, execute(_create_one_circuit(configuration, eta),
                                         backend=GLOBAL_BACKEND,
                                         shots=plays).result()).get_counts()
                    for idx, eta in enumerate(configuration['eta_group'])]

    counts_distribution = get_max_counts_distribution_for_all_channels(eta_counts, [0,0,0,0])
    return (counts_distribution[0] + 
            counts_distribution[1] + 
            counts_distribution[2] + 
            counts_distribution[3]) / (plays * len(configuration['eta_group']))

In [134]:
def get_max_counts_distribution_for_all_channels(
    all_channel_counts: List[dict], 
    counts_distribution: Tuple[float, float, float, float] = [0.0, 0.0, 0.0, 0.0]) -> Tuple[float, float, float, float]:
    """ returns the max counts between the max counts up to that moment and the circuit counts """
    if not all_channel_counts:
        return counts_distribution
    
    one_channel_counts = all_channel_counts.pop()
    max_counts = [0,0,0,0]
    if '00' in one_channel_counts: 
        max_counts[0] = max([counts_distribution[0], one_channel_counts['00']])
    if '01' in one_channel_counts: 
        max_counts[1] = max([counts_distribution[1], one_channel_counts['01']])
    if '10' in one_channel_counts: 
        max_counts[2] = max([counts_distribution[2], one_channel_counts['10']])
    if '11' in one_channel_counts: 
        max_counts[3] = max([counts_distribution[3], one_channel_counts['11']])

    return get_max_counts_distribution_for_all_channels(all_channel_counts, max_counts)

In [83]:
def _create_one_circuit(configuration: dict,
                        eta: float) -> QuantumCircuit:
    """ Creates one circuit from a given configuration and eta """
    qreg_q = QuantumRegister(3, 'q')
    creg_c = ClassicalRegister(2, 'c')

    initial_state = _prepare_initial_state_entangled(configuration['state_probability'])

    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.initialize(initial_state, [0, 1])
    circuit.reset(qreg_q[2])
    circuit.cry(2 * eta, qreg_q[1], qreg_q[2])
    circuit.cx(qreg_q[2], qreg_q[1])
    circuit.barrier()
    circuit.rx(configuration['angle_rx1'], qreg_q[1])
    circuit.ry(configuration['angle_ry1'], qreg_q[1])
    circuit.rx(configuration['angle_rx0'], qreg_q[0])
    circuit.ry(configuration['angle_ry0'], qreg_q[0])
    circuit.cx(qreg_q[0], qreg_q[1])
    circuit.h(qreg_q[0])
    circuit.measure([0, 1], creg_c)
    return circuit

In [84]:
def _prepare_initial_state_entangled(state_probability: float) -> Tuple[complex, complex, complex, complex]:
    """ Prepare initial state: computing 'y' as the amplitudes  """
    return (0, np.sqrt(state_probability), np.sqrt(1 - state_probability), 0)

In [85]:
def add_initial_parameters_and_variable_bounds(optimization_setup: dict)-> dict:
    """ add initial parameters and variable bounds to the initial optimization setup """
    updated_optimization_setup = optimization_setup
    updated_optimization_setup['initial_parameters'] = [0] * ((optimization_setup['number_channels_to_discriminate'] * 2) + 1)
    variable_bounds = [(0,1)] # amplitude_probability
    variable_bounds += [(0, 2*np.pi) for i in range(optimization_setup['number_channels_to_discriminate'] * 2)] # rx & ry for each channel
    updated_optimization_setup['variable_bounds'] = variable_bounds
    return updated_optimization_setup

## Launch Optimization

In [135]:
clone_setup=None
eta_groups_idx_to_optimize, optimal_configurations = _select_eta_groups_to_optimize(clone_setup, optimization_setup['number_channels_to_discriminate'])

updated_optimization_setup = add_initial_parameters_and_variable_bounds(optimization_setup)

print(f'number of eta_groups_idx_to_optimize: {len(eta_groups_idx_to_optimize)} -> {eta_groups_idx_to_optimize}')

program_start_time = time.time()
print("Starting the execution")
    
for eta_group_idx in eta_groups_idx_to_optimize:
    start_time = time.time()
    GLOBAL_ETA_GROUP = GLOBAL_ETA_GROUPS[eta_group_idx]
    result = _compute_best_configuration(updated_optimization_setup)
    # print(result)
    optimal_configurations['probabilities'].append(result['best_probability'])
    optimal_configurations['configurations'].append(result['best_configuration'])
    optimal_configurations['best_algorithm'].append(result['best_algorithm'])
    optimal_configurations['number_calls_made'].append(result['number_calls_made'])
    end_time = time.time()
    print(f"Group of etas # {eta_group_idx+1} of {len(eta_groups_idx_to_optimize)}, time taken this group of etas: " +
              f'{np.round((end_time - start_time)/60, 0)} minutes' +
              f' and {np.round((end_time - start_time) % 60, 0)} seconds')
    print("total minutes taken this group of etas: ", int(np.round((end_time - start_time) / 60)))
    print("total time taken so far: " +
              f'{np.round((end_time - program_start_time)/60, 0)} minutes' +
              f' and {np.round((end_time - program_start_time) % 60, 0)} seconds')
end_time = time.time()
print("total minutes of execution time: ", int(np.round((end_time - program_start_time) / 60)))
print(f'Number eta groups optimized: {len(eta_groups_idx_to_optimize)}' +
          f'from the total eta groups: {len(GLOBAL_ETA_GROUPS)} ')
optimal_configurations['eta_groups'] = GLOBAL_ETA_GROUPS
results= optimal_configurations

number of eta_groups_idx_to_optimize: 210 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209]
Starting the e

Best Average Probability: 0.975
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.975
Number of cost function calls made: 100
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 76°, η₀ = 0°
Group of etas # 17 of 210, time taken this group of etas: 0.0 minutes and 4.0 seconds
total minutes taken this group of etas:  0
total time taken so far: 1.0 minutes and 56.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.9915
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.9915
Number of cost function calls made: 102
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 81°, η₀ = 0°
Group of etas # 18 of 210, time taken this group of etas: 0.0 minutes and 4.0 seconds
total minutes taken this group of etas:  0
total time taken so far: 1.0 minutes and 59.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.999
Final Best Optimizer Alg

SystemError: <class 'enumerate'> returned a result with an error set

In [583]:
results

{'eta_groups': [(0.07853981633974483, 0.0),
  (0.15707963267948966, 0.0),
  (0.23561944901923448, 0.0),
  (0.3141592653589793, 0.0),
  (0.39269908169872414, 0.0),
  (0.47123889803846897, 0.0),
  (0.5497787143782138, 0.0),
  (0.6283185307179586, 0.0),
  (0.7068583470577035, 0.0),
  (0.7853981633974483, 0.0),
  (0.8639379797371931, 0.0),
  (0.9424777960769379, 0.0),
  (1.0210176124166828, 0.0),
  (1.0995574287564276, 0.0),
  (1.1780972450961724, 0.0),
  (1.2566370614359172, 0.0),
  (1.335176877775662, 0.0),
  (1.413716694115407, 0.0),
  (1.4922565104551517, 0.0),
  (1.5707963267948966, 0.0),
  (0.15707963267948966, 0.07853981633974483),
  (0.23561944901923448, 0.07853981633974483),
  (0.3141592653589793, 0.07853981633974483),
  (0.39269908169872414, 0.07853981633974483),
  (0.47123889803846897, 0.07853981633974483),
  (0.5497787143782138, 0.07853981633974483),
  (0.6283185307179586, 0.07853981633974483),
  (0.7068583470577035, 0.07853981633974483),
  (0.7853981633974483, 0.07853981633974

In [582]:
save_results_to_disk(results, filename)
# save_results_to_disk(results, '20210412d_C2_A1_100_10000')